In [1]:
import os, fileinput
from dotenv import load_dotenv
from ec2_management.ec2_manager import EC2Client

In [3]:
load_dotenv()
AWS_ACCESS_KEY = os.environ['AWS_LAB_ACCESS_KEY']
AWS_SECRET_ACCESS_KEY = os.environ['AWS_LAB_SECRET_ACCESS_KEY']
AWS_SESSION_TOKEN = os.environ['AWS_LAB_SESSION_TOKEN']
AWS_REGION = os.environ['AWS_REGION']
AWS_KEY_PAIR = os.environ['AWS_KEY_PAIR']

In [4]:
client = EC2Client(
    aws_access_key=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    aws_region=AWS_REGION,
    aws_key_pair=AWS_KEY_PAIR
)

In [ ]:
# add rule to the default security group to open port 8080
client.open_port(default_group=True, port_number=8080)

In [3]:
# create and run the server ec2 instance
flower_server = client.create_instance('server', startup_script_path='flower-dependencies/server/server_startup.txt')

In [12]:
# read client start-up script to modify the server public IP there
with open('flower-dependencies/client/client_startup.txt', 'r') as f:
    client_startup_script = [line for line in f.readlines()]
# modify the line
client_startup_script = [line.replace(line[line.find('= ')+len('= '):line.find('\\')], flower_server.public_ip_address) for line in client_startup_script]
# write new version of the script
with open('flower-dependencies/client/client_startup.txt', 'w') as f:
    f.writelines(client_startup_script)

In [ ]:
# create and run the clients ec2 instances
flower_clients = client.create_instance('clients', startup_script_path='flower-dependencies/client/client_startup.txt', n_instances=2)

In [3]:
client.terminate_instance(instance_ids=[i['InstanceId'] for i in flower_clients['Instances']])

{'TerminatingInstances': [{'CurrentState': {'Code': 32,
    'Name': 'shutting-down'},
   'InstanceId': 'i-0156f5f0d72cf92a0',
   'PreviousState': {'Code': 16, 'Name': 'running'}},
  {'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
   'InstanceId': 'i-0c16cf6a52199f91d',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': 'fbb6da46-51f2-4772-9dfa-35802137da00',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fbb6da46-51f2-4772-9dfa-35802137da00',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '938',
   'date': 'Fri, 17 Nov 2023 19:55:51 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}